<a href="https://colab.research.google.com/github/castillosebastian/research_tools/blob/master/deep_learning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with MNIST db

In [1]:
import os

In [2]:
os.getcwd()

'/content'